# Character-level recurrent sequence-to-sequence model

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2017/09/29<br>
**Last modified:** 2020/04/26<br>
**Description:** Character-level recurrent sequence-to-sequence model.

## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.


## Setup


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

# Instantiates the device to be used as GPU/CPU based on availability
device_gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Visualization tools
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

import random
import csv

## Download the data


In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip


['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

## Configuration


In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"


## Prepare the data


In [ ]:
input_characters =[' ']
target_characters = [' ']

In [ ]:
eng_alphabets = 'abcdefghijklmnopqrstuvwxyz'
pad_char = '-PAD-'

eng_alpha2index = {pad_char: 0}
for index, alpha in enumerate(eng_alphabets):
    eng_alpha2index[alpha] = index+1
    input_characters.append(alpha)

print(eng_alpha2index)

{'-PAD-': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [ ]:
input_characters

[' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [ ]:
import re
non_eng_letters_regex = re.compile('[^a-zA-Z ]')

# Remove all English non-letters
def cleanEnglishVocab(line):
    line = line.replace('-', ' ').replace(',', ' ').upper()
    line = non_eng_letters_regex.sub('', line)
    return line.split()

# Remove all Hindi non-letters
def cleanHindiVocab(line):
    line = line.replace('-', ' ').replace(',', ' ')
    cleaned_line = ''
    for char in line:
        if char in hindi_alpha2index or char == ' ':
            cleaned_line += char
    return cleaned_line.split()

In [ ]:
# Hindi Unicode Hex Range is 2304:2432. Source: https://en.wikipedia.org/wiki/Devanagari_(Unicode_block)

hindi_alphabets = [chr(alpha) for alpha in range(2304, 2432)]
hindi_alphabet_size = len(hindi_alphabets)

hindi_alpha2index = {pad_char: 0}
for index, alpha in enumerate(hindi_alphabets):
    hindi_alpha2index[alpha] = index+1
    target_characters.append(alpha)

print(hindi_alpha2index)

{'-PAD-': 0, 'ऀ': 1, 'ँ': 2, 'ं': 3, 'ः': 4, 'ऄ': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ऌ': 13, 'ऍ': 14, 'ऎ': 15, 'ए': 16, 'ऐ': 17, 'ऑ': 18, 'ऒ': 19, 'ओ': 20, 'औ': 21, 'क': 22, 'ख': 23, 'ग': 24, 'घ': 25, 'ङ': 26, 'च': 27, 'छ': 28, 'ज': 29, 'झ': 30, 'ञ': 31, 'ट': 32, 'ठ': 33, 'ड': 34, 'ढ': 35, 'ण': 36, 'त': 37, 'थ': 38, 'द': 39, 'ध': 40, 'न': 41, 'ऩ': 42, 'प': 43, 'फ': 44, 'ब': 45, 'भ': 46, 'म': 47, 'य': 48, 'र': 49, 'ऱ': 50, 'ल': 51, 'ळ': 52, 'ऴ': 53, 'व': 54, 'श': 55, 'ष': 56, 'स': 57, 'ह': 58, 'ऺ': 59, 'ऻ': 60, '़': 61, 'ऽ': 62, 'ा': 63, 'ि': 64, 'ी': 65, 'ु': 66, 'ू': 67, 'ृ': 68, 'ॄ': 69, 'ॅ': 70, 'ॆ': 71, 'े': 72, 'ै': 73, 'ॉ': 74, 'ॊ': 75, 'ो': 76, 'ौ': 77, '्': 78, 'ॎ': 79, 'ॏ': 80, 'ॐ': 81, '॑': 82, '॒': 83, '॓': 84, '॔': 85, 'ॕ': 86, 'ॖ': 87, 'ॗ': 88, 'क़': 89, 'ख़': 90, 'ग़': 91, 'ज़': 92, 'ड़': 93, 'ढ़': 94, 'फ़': 95, 'य़': 96, 'ॠ': 97, 'ॡ': 98, 'ॢ': 99, 'ॣ': 100, '।': 101, '॥': 102, '०': 103, '१': 104, '२': 105, '३': 106, '४': 107, '५': 108, '६': 109, '७': 

In [ ]:
target_characters

[' ',
 'ऀ',
 'ँ',
 'ं',
 'ः',
 'ऄ',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ऌ',
 'ऍ',
 'ऎ',
 'ए',
 'ऐ',
 'ऑ',
 'ऒ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'ङ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'ऩ',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ऱ',
 'ल',
 'ळ',
 'ऴ',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 'ऺ',
 'ऻ',
 '़',
 'ऽ',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'ॄ',
 'ॅ',
 'ॆ',
 'े',
 'ै',
 'ॉ',
 'ॊ',
 'ो',
 'ौ',
 '्',
 'ॎ',
 'ॏ',
 'ॐ',
 '॑',
 '॒',
 '॓',
 '॔',
 'ॕ',
 'ॖ',
 'ॗ',
 'क़',
 'ख़',
 'ग़',
 'ज़',
 'ड़',
 'ढ़',
 'फ़',
 'य़',
 'ॠ',
 'ॡ',
 'ॢ',
 'ॣ',
 '।',
 '॥',
 '०',
 '१',
 '२',
 '३',
 '४',
 '५',
 '६',
 '७',
 '८',
 '९',
 '॰',
 'ॱ',
 'ॲ',
 'ॳ',
 'ॴ',
 'ॵ',
 'ॶ',
 'ॷ',
 'ॸ',
 'ॹ',
 'ॺ',
 'ॻ',
 'ॼ',
 'ॽ',
 'ॾ',
 'ॿ']

In [ ]:
from torch.utils.data import Dataset
import xml.etree.ElementTree as ET

class TransliterationDataLoader(Dataset):
    def __init__(self, filename):
        self.eng_words, self.hindi_words = self.readXmlDataset(filename, cleanHindiVocab)
        self.shuffle_indices = list(range(len(self.eng_words)))
        random.shuffle(self.shuffle_indices)
        self.shuffle_start_index = 0
        
    def __len__(self):
        return len(self.eng_words)
    
    def __getitem__(self, idx):
        return self.eng_words[idx], self.hindi_words[idx]
    
    def readXmlDataset(self, filename, lang_vocab_cleaner):
        tsv_file = open(filename)
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        lang1_words = []
        lang2_words = []

        for row in read_tsv:
            lang2_words.append(row[0])
            lang1_words.append(row[1])
            #print(row[0])

        return lang1_words, lang2_words
    
    def get_random_sample(self):
        return self.__getitem__(np.random.randint(len(self.eng_words)))
    
    def get_batch_from_array(self, batch_size, array):
        end = self.shuffle_start_index + batch_size
        batch = []
        if end >= len(self.eng_words):
            batch = [array[i] for i in self.shuffle_indices[0:end%len(self.eng_words)]]
            end = len(self.eng_words)
        return batch + [array[i] for i in self.shuffle_indices[self.shuffle_start_index : end]]
    
    def get_batch(self, batch_size, postprocess = True):
        eng_batch = self.get_batch_from_array(batch_size, self.eng_words)
        hindi_batch = self.get_batch_from_array(batch_size, self.hindi_words)
        self.shuffle_start_index += batch_size + 1
        
        # Reshuffle if 1 epoch is complete
        if self.shuffle_start_index >= len(self.eng_words):
            random.shuffle(self.shuffle_indices)
            self.shuffle_start_index = 0
            
        return eng_batch, hindi_batch

In [ ]:
def readXmlDataset(filename):
        tsv_file = open(filename)
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        lang1_words = []
        lang2_words = []

        for row in read_tsv:
            lang2_words.append('    '+row[0])
            lang1_words.append('    '+row[1])
            #print(row[0])

        return lang1_words, lang2_words

In [ ]:
input_texts, target_texts = readXmlDataset('hi.translit.sampled.train.tsv')


In [ ]:
input_texts

['    an',
 '    ankganit',
 '    uncle',
 '    ankur',
 '    ankuran',
 '    ankurit',
 '    aankush',
 '    ankush',
 '    ang',
 '    anga',
 '    agandh',
 '    angad',
 '    angane',
 '    angbhang',
 '    angarakshak',
 '    angrakshak',
 '    angara',
 '    angaare',
 '    angare',
 '    angi',
 '    angikar',
 '    anguthe',
 '    angul',
 '    anguliyon',
 '    anguli',
 '    ungli',
 '    angutha',
 '    aanguthiyon',
 '    anguthiyon',
 '    anguthi',
 '    anguthe',
 '    anguthon',
 '    angoor',
 '    angur',
 '    anguri',
 '    anguron',
 '    anguuron',
 '    angon',
 '    angola',
 '    angraji',
 '    angreji',
 '    angrezi',
 '    angreji',
 '    anchal',
 '    anchalon',
 '    anchlon',
 '    anjan',
 '    anjana',
 '    anjali',
 '    anjaan',
 '    anjan',
 '    anjaani',
 '    anjani',
 '    anjaane',
 '    anjane',
 '    anjar',
 '    anjeer',
 '    anjum',
 '    anjuman',
 '    anju',
 '    antarctica',
 '    antartica',
 '    antaniyo',
 '    antonio',
 '   

In [ ]:
# Vectorize the data.
'''input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))'''
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 44204
Number of unique input tokens: 27
Number of unique output tokens: 129
Max sequence length for inputs: 24
Max sequence length for outputs: 23


In [ ]:
target_characters 

[' ',
 'ऀ',
 'ँ',
 'ं',
 'ः',
 'ऄ',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ऌ',
 'ऍ',
 'ऎ',
 'ए',
 'ऐ',
 'ऑ',
 'ऒ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'ङ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'ऩ',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ऱ',
 'ल',
 'ळ',
 'ऴ',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 'ऺ',
 'ऻ',
 '़',
 'ऽ',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'ॄ',
 'ॅ',
 'ॆ',
 'े',
 'ै',
 'ॉ',
 'ॊ',
 'ो',
 'ौ',
 '्',
 'ॎ',
 'ॏ',
 'ॐ',
 '॑',
 '॒',
 '॓',
 '॔',
 'ॕ',
 'ॖ',
 'ॗ',
 'क़',
 'ख़',
 'ग़',
 'ज़',
 'ड़',
 'ढ़',
 'फ़',
 'य़',
 'ॠ',
 'ॡ',
 'ॢ',
 'ॣ',
 '।',
 '॥',
 '०',
 '१',
 '२',
 '३',
 '४',
 '५',
 '६',
 '७',
 '८',
 '९',
 '॰',
 'ॱ',
 'ॲ',
 'ॳ',
 'ॴ',
 'ॵ',
 'ॶ',
 'ॷ',
 'ॸ',
 'ॹ',
 'ॺ',
 'ॻ',
 'ॼ',
 'ॽ',
 'ॾ',
 'ॿ']

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        print(char)
    if i==5:
        break

 
 
 
 
a
n
 
 
 
 
a
n
k
g
a
n
i
t
 
 
 
 
u
n
c
l
e
 
 
 
 
a
n
k
u
r
 
 
 
 
a
n
k
u
r
a
n
 
 
 
 
a
n
k
u
r
i
t


In [ ]:
print(input_texts)

['    an', '    ankganit', '    uncle', '    ankur', '    ankuran', '    ankurit', '    aankush', '    ankush', '    ang', '    anga', '    agandh', '    angad', '    angane', '    angbhang', '    angarakshak', '    angrakshak', '    angara', '    angaare', '    angare', '    angi', '    angikar', '    anguthe', '    angul', '    anguliyon', '    anguli', '    ungli', '    angutha', '    aanguthiyon', '    anguthiyon', '    anguthi', '    anguthe', '    anguthon', '    angoor', '    angur', '    anguri', '    anguron', '    anguuron', '    angon', '    angola', '    angraji', '    angreji', '    angrezi', '    angreji', '    anchal', '    anchalon', '    anchlon', '    anjan', '    anjana', '    anjali', '    anjaan', '    anjan', '    anjaani', '    anjani', '    anjaane', '    anjane', '    anjar', '    anjeer', '    anjum', '    anjuman', '    anju', '    antarctica', '    antartica', '    antaniyo', '    antonio', '    antoni', '    antony', '    andaman', '    under', '    underco

In [ ]:
input_texts

['    an',
 '    ankganit',
 '    uncle',
 '    ankur',
 '    ankuran',
 '    ankurit',
 '    aankush',
 '    ankush',
 '    ang',
 '    anga',
 '    agandh',
 '    angad',
 '    angane',
 '    angbhang',
 '    angarakshak',
 '    angrakshak',
 '    angara',
 '    angaare',
 '    angare',
 '    angi',
 '    angikar',
 '    anguthe',
 '    angul',
 '    anguliyon',
 '    anguli',
 '    ungli',
 '    angutha',
 '    aanguthiyon',
 '    anguthiyon',
 '    anguthi',
 '    anguthe',
 '    anguthon',
 '    angoor',
 '    angur',
 '    anguri',
 '    anguron',
 '    anguuron',
 '    angon',
 '    angola',
 '    angraji',
 '    angreji',
 '    angrezi',
 '    angreji',
 '    anchal',
 '    anchalon',
 '    anchlon',
 '    anjan',
 '    anjana',
 '    anjali',
 '    anjaan',
 '    anjan',
 '    anjaani',
 '    anjani',
 '    anjaane',
 '    anjane',
 '    anjar',
 '    anjeer',
 '    anjum',
 '    anjuman',
 '    anju',
 '    antarctica',
 '    antartica',
 '    antaniyo',
 '    antonio',
 '   

## Build the model


In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


## Train the model


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


Epoch 1/100
553/553 [==============================] - 40s 12ms/step - loss: 1.1818 - accuracy: 0.7362 - val_loss: 1.0723 - val_accuracy: 0.7506
Epoch 2/100
553/553 [==============================] - 6s 10ms/step - loss: 0.7837 - accuracy: 0.7858 - val_loss: 0.8920 - val_accuracy: 0.7781
Epoch 3/100
553/553 [==============================] - 6s 10ms/step - loss: 0.4953 - accuracy: 0.8523 - val_loss: 0.6651 - val_accuracy: 0.8166
Epoch 4/100
553/553 [==============================] - 6s 11ms/step - loss: 0.3399 - accuracy: 0.8929 - val_loss: 0.5693 - val_accuracy: 0.8371
Epoch 5/100
553/553 [==============================] - 6s 11ms/step - loss: 0.2631 - accuracy: 0.9163 - val_loss: 0.5412 - val_accuracy: 0.8470
Epoch 6/100
553/553 [==============================] - 6s 10ms/step - loss: 0.2169 - accuracy: 0.9305 - val_loss: 0.5043 - val_accuracy: 0.8586
Epoch 7/100
553/553 [==============================] - 6s 10ms/step - loss: 0.1847 - accuracy: 0.9405 - val_loss: 0.4781 - val_accuracy

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states


In [ ]:


# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[' ']] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence



You can now generate decoded sentences as such:


In [ ]:
randomlist = []
for i in range(0,10):
    n = random.randint(100,1000)
    randomlist.append(n)
print(randomlist)

[730, 376, 173, 406, 418, 736, 733, 209, 851, 455]


In [ ]:
for seq_index in randomlist:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence:     adheer
Decoded sentence:    अधीर                 
-
Input sentence:     agrasar
Decoded sentence:    अग्रसर               
-
Input sentence:     andhkaar
Decoded sentence:    अंधकार               
-
Input sentence:     ahet
Decoded sentence:    अचेत                 
-
Input sentence:     achhi
Decoded sentence:    अछी                  
-
Input sentence:     adhuura
Decoded sentence:    अधूरा                
-
Input sentence:     adhirata
Decoded sentence:    अधीरता               
-
Input sentence:     anshakalik
Decoded sentence:    अंशकालिक             
-
Input sentence:     anaar
Decoded sentence:    अनार                 
-
Input sentence:     ajava
Decoded sentence:    अजवा                 


In [ ]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)


-
Input sentence:     an
Decoded sentence:    अं                   
-
Input sentence:     ankganit
Decoded sentence:    अंकगणि               
-
Input sentence:     uncle
Decoded sentence:    अंकल                 
-
Input sentence:     ankur
Decoded sentence:    अंकुर                
-
Input sentence:     ankuran
Decoded sentence:    अंकुरण               
-
Input sentence:     ankurit
Decoded sentence:    अंकुरित              
-
Input sentence:     aankush
Decoded sentence:    अंकुश                
-
Input sentence:     ankush
Decoded sentence:    अंकुश                
-
Input sentence:     ang
Decoded sentence:    अंग                  
-
Input sentence:     anga
Decoded sentence:    अंग                  
-
Input sentence:     agandh
Decoded sentence:    अंगद                 
-
Input sentence:     angad
Decoded sentence:    अंगद                 
-
Input sentence:     angane
Decoded sentence:    अंगने                
-
Input sentence:     angbhang
Decoded sentence:    अंगभंग             